In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np

In [2]:
data = pd.read_csv("Cleaned_data.csv")

In [3]:
lb_amc = LabelEncoder()
data.Amc_Name = lb_amc.fit_transform(data.Amc_Name)
unique_Amc_Nos = data.Amc_Name.unique()
unique_Amc_labels = lb_amc.inverse_transform(data.Amc_Name.unique())
amc_dict = {} # to store the mapping of the label encoding to the amc name
for i in range(len(unique_Amc_labels)):
    amc_dict[unique_Amc_labels[i]] = unique_Amc_Nos[i]

In [4]:
lb_crop = LabelEncoder()
data.Crop = lb_crop.fit_transform(data.Crop)
unique_Crop_Nos = data.Crop.unique()
unique_Crop_labels = lb_crop.inverse_transform(unique_Crop_Nos)
crop_dict = {} # to store the mapping of the label encoding to the crop name
for i in range(len(unique_Crop_labels)):
    crop_dict[unique_Crop_labels[i]] = unique_Crop_Nos[i]

In [5]:
crop_max = data.Crop.max()
crop_min = data.Crop.min()
amc_max = data.Amc_Name.max()
amc_min = data.Amc_Name.min()
max_max = data.Maximum.max()
max_min = data.Maximum.min()
min_max = data.Minimum.max()
min_min = data.Minimum.min()
modal_max = data.Modal.max()
modal_min = data.Modal.min()

In [6]:
model = load_model("price_predict_model.h5")

In [12]:
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify
app =Flask(__name__)
run_with_ngrok(app) #starts ngrok when the app is running
@app.route("/<market_label>/<crop_label>/<int:maximum>/<int:minimum>")
def home(market_label,crop_label,maximum,minimum):
    market = amc_dict.get(market_label)
    crop = crop_dict.get(crop_label)
    print("Input AMC Name: ",market_label)
    print("Input Crop: ",crop_label)
    print("Input Maximum: ",maximum)
    print("Input Minimum: ",minimum)
    
    market_scale = np.round(((market - amc_min)/(amc_max- amc_min)),4)
    crop_scale = np.round(((crop - crop_min)/(crop_max-crop_min)),4)
    maximum_scale = np.round(((maximum - max_min)/(max_max-max_min)),4)
    minimum_scale = np.round(((minimum - min_min)/(min_max-min_min)),4)

    X = [market_scale,crop_scale, maximum_scale, minimum_scale]
    X = np.array(X).reshape(-1,4)
    X = X.reshape((X.shape[0], 1,X.shape[1]))

    Y_pred = model.predict(X)

    Y_final = Y_pred*(maximum-minimum)+minimum
    print("Output Price prediction: ",str(Y_final[0][0]))
    return jsonify(str(Y_final[0][0]))
app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6c477723add7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [05/Apr/2021 11:12:25] "GET /Tandur/Groundnut%20pods-Common/6120/5550 HTTP/1.1" 200 -


Input AMC Name:  Tandur
Input Crop:  Groundnut pods-Common
Input Maximum:  6120
Input Minimum:  5550
Output Price prediction:  5987.5317


127.0.0.1 - - [05/Apr/2021 11:12:37] "GET /Narayanpet/Bengal%20Gram-Common/4614/4565 HTTP/1.1" 200 -


Input AMC Name:  Narayanpet
Input Crop:  Bengal Gram-Common
Input Maximum:  4614
Input Minimum:  4565
Output Price prediction:  4594.89


127.0.0.1 - - [05/Apr/2021 11:12:41] "GET /Warangal/Maize-Common/1381/1288 HTTP/1.1" 200 -


Input AMC Name:  Warangal
Input Crop:  Maize-Common
Input Maximum:  1381
Input Minimum:  1288
Output Price prediction:  1305.0516
